# Generate tiled region run file

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: Before running this notebook for the first time, make sure you've coregistered your instrument using the *update coregistration parameters* section of  `1_set_up_toffy.ipynb`. This will ensure your FOVs display correctly on the slide.

This notebook automatically generates FOVs across multiple tiled regions on a sample and allows you to remove unneeded FOVs created during the process.

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import json
import os
from skimage.io import imread

from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

## Define the data paths

### Preset the *json_tiling_dir*, the directory to place your input files in and write tiling info to, **do not change this variable**.

In [ ]:
json_tiling_dir = 'C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons'

### Define a name prefix to use for the input and output files to place in *json_tiling_dir*. **All files must conform to this naming scheme**.

In [ ]:
# define the prefix for each file
tiled_region_prefix = 'example_tiled_region'

### Place *region_corners_path*, the JSON file which defines each tiled region with their corresponding top-left corner FOVs, directly in *json_tiling_dir*. To construct, download the run JSON file for the desired MIBI run off the CAC.

### In the *'fovs'* list of that file, keep only FOVs you wish to define as a top-left corner for a distinct region. Each FOV should contain keys *'fovSizeMicrons'* (distance between FOV centroids along row and column axes), *'centerPointMicrons'* (centroid of the top-left FOV), and *'name'* (name of this tiled region). For each FOV, manually rename the *'name'* key to the desired name of the corresponding region that FOV defines.

### Double check that the FOV parameters defined in *region_corners_path* are consistent with their corresponding run values. For example, if a FOV was processed with size 400 by the run, its *fovSizeMicrons* should also be 400.

In [ ]:
region_corners_path = os.path.join(json_tiling_dir, '%s_region_corners.json' % tiled_region_prefix) 

### Place *slide_path*, the PNG file containing the slide to visualize the tiled regions over, directly in *json_tiling_dir*.

In [ ]:
slide_path = os.path.join(json_tiling_dir, "%s_slide.png" % tiled_region_prefix)

### Place *moly_path*, the JSON file defining a Moly point, directly in *json_tiling_dir*. This is needed if you want to insert Moly points at certain intervals between FOVs for calibration purposes.

In [ ]:
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tiled_region_prefix)

### Define *tiled_region_path*, the output file in *json_tiling_dir* to write the final set of FOVs for each tiled region after the notebook is complete.

In [ ]:
tiled_region_path = os.path.join(json_tiling_dir, "sample_tiled_region_mapping.json")

#### When starting up your run, ensure the directory structure of `json_tiling_dir` looks like:

- `json_tiling_dir`
    - `region_corners_path`
    - `slide_path`
    - `moly_path`

## Set tiling parameters

### Run following cell, which will ask you to enter additional info for each specified region in *region_corners_path*. This will include the number of FOVs along the row and column axes, whether to randomize the order of FOVs in a tiled region, whether to insert Moly points between tiled regions, and whether to insert Moly points between FOVs at a specified interval. 

#### Refer to this link for how Moly point insertion works: https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.mibi.html#ark.mibi.tiling_utils.tiled_region_generate_fov_list

In [ ]:
tiling_params = tiling_utils.set_tiled_region_params(region_corners_path)

## Create the FOVs for each region

### FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed. For example, *R1C1* means the FOV is in row 1 and column 1, *R2C1* would be the next FOV down, and *R1C2* would be the next FOV to the right. The top left FOV is *R1C1*, the bottom right FOV would be *R4C5* for a tiled region of 4x5 dimensions. 

In [ ]:
# generate the FOVs in each region
tiled_region_fovs = tiling_utils.generate_tiled_region_fov_list(
    tiling_params,
    moly_path
)

## Interactive FOV deselection

### The grid defining an ROI may cover more area than you wish to tile. This GUI allows you to interactively delete FOVs from `tiled_region_fovs` prior to saving:

### 1. Click on a FOV to propose it for removal. The border of proposed FOVs will be highlighted. Click again to unpropose the FOV.
### 2.Once you've selected all the FOVs you want to remove, click *Delete selected FOVs* to remove them. **This action cannot be undone!**
### 3. You can always remove more FOVs even after saving. Just remember to re-save.
### 4. After you're done removing all the FOVs, click *Save mapping* to save your truncated set of FOVs to *tiled_region_path*.

In [ ]:
# read the slide image in
slide_data = imread(slide_path)

In [ ]:
%matplotlib widget
_ = tiling_utils.tiled_region_interactive_remap(
    tiled_region_fovs,
    tiling_params,
    slide_data,
    tiled_region_path,
    figsize=(7, 7)
)